In [2]:
#shamelesly stealing https://www.tensorflow.org/text/tutorials/classify_text_with_bert
import bert_handle as bert
import tensorflow_hub as hub
import tensorflow_text

In [3]:
bert_stuff=bert.get_bert(bert.bert_model_name)

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [4]:
bert_preprocess_model = hub.KerasLayer(bert_stuff["tfhub_handle_preprocess"])
bert_model = hub.KerasLayer(bert_stuff["tfhub_handle_encoder"])

In [5]:
text_test = ['Verizon Communications (VZ) Gains As Market Dips: What You Should Know ',"other text"]
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_type_ids', 'input_word_ids']
Shape      : (2, 128)
Word Ids   : [  101  2310 21885  2239  4806  1006  1058  2480  1007 12154  2004  3006]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [6]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {bert_stuff["tfhub_handle_encoder"]}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(2, 512)
Pooled Outputs Values:[ 0.87515324  0.9615815   0.12347151  0.32359517  0.05750871  0.9694411
  0.9846227  -0.79250544 -0.10360488 -0.99781424 -0.12245988 -0.985213  ]
Sequence Outputs Shape:(2, 128, 512)
Sequence Outputs Values:[[ 0.46611235 -0.23308545  1.291422   ... -0.76153916  0.32311335
  -0.219411  ]
 [ 0.41399637 -0.7157209  -0.2752384  ... -0.31453577  0.8399521
  -0.98781025]
 [ 0.17573708  0.6077015   0.02832541 ... -0.5685752   0.7769697
   0.7286631 ]
 ...
 [ 0.35955298  0.37087652  0.6157819  ... -0.38470975  0.9454852
   0.52486944]
 [ 0.14542109  0.88189965  0.29846963 ... -0.6534967  -0.5762236
  -0.4375792 ]
 [-0.5927848   0.15682581  0.13009605 ... -0.19035406  0.29293072
  -0.5804826 ]]


In [33]:
bert_results.keys()

dict_keys(['pooled_output', 'default', 'encoder_outputs', 'sequence_output'])

In [35]:
bert_results["pooled_output"]

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[ 0.87515324,  0.9615815 ,  0.12347151, ...,  0.08707284,
        -0.27692407, -0.80007094],
       [ 0.9955232 , -0.7738685 ,  0.26882803, ..., -0.11124813,
         0.16591723, -0.6607704 ]], dtype=float32)>

In [11]:
import numpy as np 
import os
from os.path import join

import shutil

In [19]:
stocks=os.listdir("archive")

['T', 'TMUS', 'VZ']

In [15]:
for folder,subs,files in os.walk("archive"):
    if(len(files)):
        #print(f"{folder}")
        with open(join(folder,"headline"),"rb") as f:
            headline=f.read()
        with open(join(folder,"summary"),"rb") as f:
            summary=f.read()
        
        addres=join("output",folder)
        os.makedirs(addres,exist_ok=True)
        if(len(os.listdir(addres))==0):
            output=bert_model(bert_preprocess_model([headline,summary]))
            output=output["pooled_output"].numpy()
            np.save(join(addres,"bert"),output)
print("done")

done


In [12]:
#copied from stack overflow
def remove_empty_folders(path_abs):
    walk = list(os.walk(path_abs))
    for path, _, _ in walk[::-1]:
        if len(os.listdir(path)) == 0:
            shutil.rmtree(path)
remove_empty_folders("output")

In [22]:
shutil.make_archive("bert_embedings","zip",root_dir=join("output","archive"))

'C:\\Users\\Owner\\Desktop\\stock_bot2\\bert_embedings.zip'